In [1]:
using Random

### Auxilliary Functions

In [2]:
# actions coded as :left => 1, :down => 2, :right => 3, :up => 4
actions = Dict(1 => [0,-1], 2 => [1,0], 3 => [0,1], 4 => [-1,0]);
arrows = Dict(1 => '⇐', 2 => '⇓', 3 => '⇒', 4 => '⇑');
rewards = Dict('S' => -0.05, 'G' => 1.0, 'H' => -1.0, 'F' => -0.05);

grid3x3= ['S' 'F' 'F';
          'F' 'F' 'H';
          'F' 'F' 'G'];

grid4x4= ['S' 'F' 'F' 'F';
          'F' 'H' 'F' 'H';
          'F' 'F' 'F' 'H';
          'H' 'F' 'F' 'G'];

function get_grid(dim, p_holes, seed = 234)
    Random.seed!(seed)
    grid = [rand() < p_holes ? 'H' : 'F' for i in 1:dim, j in 1:dim]
    grid[1,1] = 'S'
    grid[end,end] = 'G'
    return grid
end;

function transition_matrix(grid, actions = actions)
    T = zeros(length(grid),length(actions),length(grid))
    i2s = CartesianIndices(grid)
    s2i = LinearIndices(grid)
    for i = 1:length(grid) # 
        if !(grid[i] == 'H' || grid[i] == 'G')
            index = i2s[i]
            for j = 1:length(actions)
                indices = Tuple(index) .+ actions[j]
                if all(in.( indices, (1:size(grid,1), 1:size(grid,2)))) 
                    k = s2i[indices...]
                    T[k,j,i] += 0.8
                    if actions[j][1] == 0
                        for l in [-1,1]
                            ind = Tuple(index) .+ (l,0)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2)))) 
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    else
                        for l in [-1,1]
                            ind = Tuple(index) .+ (0,l)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2))))
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    end
                else
                    T[i,j,i] += 0.8
                    if actions[j][1] == 0
                        for l in [-1,1]
                            ind = Tuple(index) .+ (l,0)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2)))) 
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    else
                        for l in [-1,1]
                            ind = Tuple(index) .+ (0,l)
                            if all(in.( ind, (1:size(grid,1), 1:size(grid,2)))) 
                                T[s2i[ind...],j,i] += 0.1
                            else
                                T[i,j,i] += 0.1
                            end
                        end
                    end
                end
            end
        end
    end
    return T
end;

function reward_matrix(grid, rewards = rewards)
    R = zeros(size(grid))
    for i = 1:length(grid)
        R[i] = rewards[grid[i]]
    end
    return R
end;


random_policy(grid,actions = actions) = rand(1:length(actions),size(grid));


function print_policy(P, grid, arrows = arrows)
    Policy = rand(Char,size(grid))
    for i = 1:length(grid)
        if grid[i] == 'F' || grid[i] == 'S' 
            Policy[i] = arrows[P[i]]
        elseif grid[i] == 'H' 
            Policy[i] = '⦷'
        else
            Policy[i] = grid[i]
        end
    end
    return Policy
end;

### Policy Evaluation Functions

In [3]:
# nagroda dla stanu s w iteracji i = 
# nagroda pierwotna dla stanu s + 
# beta * (srednia nagroda ze wszystkich stanów, 
#         do których możemy przejść ze stanu s, wzięta z poprzeniej iteracji i-1)
function evaluate!(P, v, v₁, R, T, β)
    for s = 1:length(v)
        v₁[s]= R[s] + β * sum(v .*  T[:,P[s],s])
    end
end;

function evaluate_policy(grid,P; β = 0.999, ϵ=0.0001, actions = actions)
    iter = 0
    T = transition_matrix(grid)
    R = reward_matrix(grid)
    v₁ = zeros(length(grid))
    while true
        iter += 1
        v = deepcopy(v₁)
        evaluate!(P, v, v₁, R, T, β)
        #@info v₁
        δ = maximum(abs.(v₁ - v)) 
        δ < ϵ * (1 - β) / β && break 
    end
    return v₁
end;

### Policy Iteration Algorithm


In [4]:
function improve_policy!(v, T, P, actions = actions)
    for s = 1:length(v)
        actions_vector = zeros(length(actions))
        for a = 1:length(actions)
            actions_vector[a] = sum(v .* T[:,a,s])
        end
        action = argmax(actions_vector)
        action != P[s] && (P[s] = action)
    end
end;

function policy_iteration(grid,β = 0.999, ϵ=0.0001)
    iter = 1
    T = transition_matrix(grid)
    R = reward_matrix(grid)
    v₁ = zeros(length(grid))
    P = random_policy(grid)
    while true
        iter += 1
        v = deepcopy(v₁)
        evaluate!(P, v, v₁, R, T, β)
        δ = maximum( abs.(v₁ - v)) 
        δ < ϵ * (1 - β) / β && break 
        improve_policy!(v₁, T, P)
    end 
    println("Iterations: $(iter)")
    return reshape(v₁,size(grid)),  print_policy(P, grid)
end;

In [5]:
vₚ, pₚ = policy_iteration(grid3x3);

Iterations: 45


In [6]:
vₚ

3×3 Matrix{Float64}:
 0.700269  0.646294   0.407308
 0.770455  0.721464  -1.0
 0.840105  0.912426   1.0

In [7]:
pₚ

3×3 Matrix{Char}:
 '⇓'  '⇐'  '⇐'
 '⇓'  '⇐'  '⦷'
 '⇒'  '⇒'  'G'

### Brute force algorithm

In [8]:
grid_sel = deepcopy(grid3x3);

In [9]:
grid_length = length(grid_sel);    # liczba komórek w siatcy

values = [1, 2, 3, 4];
iter = 0;

best_policy = ones(Int, grid_length);
best_policy_val = ones(Int, grid_length);
best_policy_sum_val = 0

for i in values, j in values, k in values, l in values, m in values, 
    n in values, o in values, p in values, q in values   # tutaj jak było by 4x4 to bylo by 16 pętli
    
    iter += 1
    
    if divrem(iter, 25000)[2] == 0
        println("Interations: $(iter)")
    end
    
    policy = [i, j, k, l, m, n, o, p, q]             # zbiór kierunków ruchu
    policy_val = evaluate_policy(grid_sel, policy)   # ewaluacja całej strategii
    policy_sum_val = sum(policy_val)                 # wartość całkowita rozważanej strategii,
    
    if policy_sum_val > best_policy_sum_val          # którą zmierzamy zmaksymalizować
        best_policy = policy
        best_policy_val = policy_val
        best_policy_sum_val = policy_sum_val
    end
end

Interations: 25000
Interations: 50000
Interations: 75000
Interations: 100000
Interations: 125000
Interations: 150000
Interations: 175000
Interations: 200000
Interations: 225000
Interations: 250000


In [10]:
best_policy_brute = reshape(best_policy, size(grid_sel));
println("Iterations: $(iter)")
print_policy(best_policy_brute, grid_sel)

Iterations: 262144


3×3 Matrix{Char}:
 '⇓'  '⇐'  '⇐'
 '⇓'  '⇐'  '⦷'
 '⇒'  '⇒'  'G'

### Discussion

W zadaniu domowym zostały porównane 2 metody do rozwiązania gry Frozen Lake: podejścia programowania dynamicznego pokazane na zajęciach oraz algorytm wyczerpujący, zaproponowany przeze mnie. Dla przykładu została rozważona gra o wymiarach 3 na 3 (o przyczynie tego później).

W przypadku programowania dynamicznego dla macierzy gry 3 × 3 algorytm zbiegł do rozwiązania optymalnego za 45 iteracji (1.32 sekund na mojej maszynie, AMD Ryzen 7 4800H Radeon Graphics 2.90 GHz, 32 RAM, Windows 10 x64).

Natomiast w przypadku algorytmu brute force, rozwiązanie najlepsze zostało znaleziono za (262 144) iteracji (14 m 47 sekund na mojej maszynie). 

Z tego wynika oczywista decyzja na rozpatrzenie mniejszej siatki gry. W tym problemie algorytm brute force ma złożoność obliczeniową O(4^n²) — rozważamy tyłe unikalnych kombinacji komórek siatki. W przypadku siatki 3x3 jest to liczba 4⁽³ˣ³⁾ = 4⁹ = (262 144). W przypadku siatek 4 × 4 i 8 × 8 wymagane będzie (4 294 967 296) i 3.402+38 liczby iteracji odpowiednio. 

Możemy też obliczyć czas trwania 1 iteracji. Skoro (262 144) iteracji były osiągnięte za 14 m 47 sec (887 sec), długość trwania jednej iteracji jest równa 0.003 sec. Oznacza to, że zakładając ten sam czas dla 1 iteracji, rozwiązanie dla siatki 4 × 4 byłoby znaleziono za 0.003x4294967296=1.45e7 sec (115 dni). Dla siatki 8 × 8 jest to liczba 3.17e+28 lat! Dla porównania Wszechświat istnieje 13.787e9 lat! (13.787 billionów)

Dlatego konieczne jest zastosowanie innych metod poszukiwania rozwiązania problemu. Na szczęście, grę Frozen Lake można rozbić na podproblemy powtarzające i skorzystać z zasady optymalności Belmana. 

Zauważymy też, że otrzymaliśmy te same rozwiązania zarówno w podejściu optymizacji dynamicznej, jak i przypadku exhaustive search, co jest zgodne z założeniem teoretycznym o gwarantowanej zbieżności rozwiązania otrzymanego za pomocą optymalizacji dynamicznej. 